<h1>Preprocessing<h1/>

In [1]:
import pandas as pd
from data.new_or_used import build_dataset

<h2> 1. Loading Dataset<h2/>

In [2]:
X_train, y_train, X_test, y_test = build_dataset()
df = pd.DataFrame.from_records(X_train)
df.columns

Index(['seller_address', 'warranty', 'sub_status', 'condition',
       'seller_contact', 'deal_ids', 'base_price', 'shipping',
       'non_mercado_pago_payment_methods', 'seller_id', 'variations',
       'location', 'site_id', 'listing_type_id', 'price', 'attributes',
       'buying_mode', 'tags', 'listing_source', 'parent_item_id',
       'coverage_areas', 'category_id', 'descriptions', 'last_updated',
       'international_delivery_mode', 'pictures', 'id', 'official_store_id',
       'differential_pricing', 'accepts_mercadopago', 'original_price',
       'currency_id', 'thumbnail', 'title', 'automatic_relist', 'date_created',
       'secure_thumbnail', 'stop_time', 'status', 'video_id',
       'catalog_product_id', 'subtitle', 'initial_quantity', 'start_time',
       'permalink', 'geolocation', 'sold_quantity', 'available_quantity'],
      dtype='object')

<h2> 2. Loading cleaning functions from data exploration notebook<h2/>

In [3]:
from src.data_cleaning import remove_columns, convert_list_into_len, get_non_mercado_pago_info, get_shipping_mode_info
from src.data_cleaning import col_to_remove, col_to_len

In [4]:
df = (df.pipe(remove_columns, col_to_remove).
        pipe(convert_list_into_len, col_to_len).
        pipe(get_non_mercado_pago_info).
        pipe(get_shipping_mode_info)
     )
df

,warranty,condition,listing_type_id,price,buying_mode,tags,category_id,accepts_mercadopago,title,automatic_relist,initial_quantity,sold_quantity,available_quantity,num_pic,num_var,num_att,non_mercado_pago,shipping_mode
0,None,new,bronze,80.00,buy_it_now,[dragged_bids_and_visits],MLA126406,True,Auriculares Samsung Originales Manos Libres Ca...,False,1,0,1,2,0,0,"[Transferencia bancaria, Acordar con el compra...",not_specified
1,NUESTRA REPUTACION,used,silver,2650.00,buy_it_now,[],MLA10267,True,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,1,0,1,6,0,0,"[Transferencia bancaria, Efectivo]",me2
2,None,used,bronze,60.00,buy_it_now,[dragged_bids_and_visits],MLA1227,True,"Antigua Revista Billiken, N° 1826, Año 1954",False,1,0,1,1,0,0,"[Transferencia bancaria, Tarjeta de crédito, E...",me2
3,None,new,silver,580.00,buy_it_now,[],MLA86345,True,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,1,0,1,2,0,0,"[Transferencia bancaria, Tarjeta de crédito, E...",me2
4,MI REPUTACION.,used,bronze,30.00,buy_it_now,[dragged_bids_and_visits],MLA41287,True,Serenata - Jennifer Blake,False,1,0,1,2,0,0,"[Transferencia bancaria, Acordar con el compra...",not_specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,Sin garantía,used,bronze,68.00,buy_it_now,[dragged_bids_and_visits],MLA1227,True,El Fin De Las Libertades - Benegas Lynch (h) -...,False,1,0,1,1,0,0,[N/a],me2
89996,None,new,bronze,126.00,buy_it_now,[dragged_bids_and_visits],MLA45559,True,Honda Wave Guardabarro Interior Trasero,False,1,1,1,3,0,0,"[Transferencia bancaria, Efectivo]",custom
89997,None,new,bronze,300.00,buy_it_now,[dragged_bids_and_visits],MLA81061,True,My Little Pony Completa Latino 4 Temporadas,False,1,0,1,1,0,0,[Efectivo],not_specified
89998,La garantia solo responde en caso de fallas de...,new,bronze,696.58,buy_it_now,[dragged_bids_and_visits],MLA48851,True,Accidente Cerebrovascular En La Infancia Y Ado...,False,100,0,100,1,0,0,[Acordar con el comprador],not_specified


<h2> 3. Converting Categorical into Numerical variables<h2/>

### 3.1 Converting column condition

In [5]:
df["condition"].value_counts(dropna=False)

new     48352
used    41648
Name: condition, dtype: int64

In [6]:
df["condition"] = df["condition"].map({"new": 0, "used":1})

### 3.2 Converting column listing_type_id

In [7]:
df["listing_type_id"].value_counts(dropna=False)

bronze          56904
free            19260
silver           8195
gold_special     2693
gold             2170
gold_premium      765
gold_pro           13
Name: listing_type_id, dtype: int64

In [8]:
df["listing_type_id"] = df["listing_type_id"].map({"free":0, "bronze":1, "silver":2, "gold":3, "gold_special":4, "gold_premium":5, "gold_pro":6})

### 3.3 Converting column buying_mode

In [9]:
df["buying_mode"].value_counts(dropna=False)

buy_it_now    87311
classified     1982
auction         707
Name: buying_mode, dtype: int64

In [10]:
df = df.join(pd.get_dummies(df.buying_mode, prefix='buy_mode.'))
df.drop(["buying_mode"], axis=1, inplace=True)

### 3.4 Converting column tags

In [11]:
df["tags"].value_counts(dropna=False)

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[dragged_bids_and_visits]                            65315
[]                                                   22412
[good_quality_thumbnail, dragged_bids_and_visits]     1191
[dragged_visits]                                       464
[good_quality_thumbnail]                               346
[dragged_visits, free_relist]                          259
[poor_quality_thumbnail, dragged_bids_and_visits]       10
[poor_quality_thumbnail]                                 3
Name: tags, dtype: int64

In [12]:
df = df.join(pd.json_normalize(df.pop("tags").apply(lambda x: {el: 1 for el in x})).fillna(0).add_prefix("tag."))

### 3.5 Converting column category_id

In [13]:
vc = df["category_id"].value_counts(dropna=False)
vc[vc<400]

MLA41269     399
MLA15204     395
MLA3361      389
MLA2032      386
MLA85960     344
            ... 
MLA125983      1
MLA376842      1
MLA61368       1
MLA17225       1
MLA70363       1
Name: category_id, Length: 10483, dtype: int64

In [14]:
others_list = vc[vc<400].index.to_list()

In [15]:
df["category_id"] = df["category_id"].apply(lambda x: "others" if x in others_list else x)

In [16]:
df["category_id"].value_counts(dropna=False)

others      80603
MLA1227      4139
MLA2044      1759
MLA41287      829
MLA3530       685
MLA2038       601
MLA15171      522
MLA15328      446
MLA1383       416
Name: category_id, dtype: int64

In [17]:
df = df.join(pd.get_dummies(df.pop("category_id"), prefix='cat.'))

### 3.6 Converting column accepts_mercadopago

In [18]:
df["accepts_mercadopago"].value_counts(dropna=False)

True     88018
False     1982
Name: accepts_mercadopago, dtype: int64

In [19]:
df["accepts_mercadopago"] = df["accepts_mercadopago"].apply(lambda x: int(x))

### 3.7 Converting column automatic_relist

In [20]:
df["automatic_relist"].value_counts(dropna=False)

False    85773
True      4227
Name: automatic_relist, dtype: int64

In [21]:
df["automatic_relist"] = df["automatic_relist"].apply(lambda x: int(x))

### 3.8 Converting column non_mercado_pago

In [22]:
df["non_mercado_pago"].value_counts(dropna=False)

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[N/a]                                                                                                                                          27531
[Transferencia bancaria, Efectivo]                                                                                                             20303
[Transferencia bancaria, Tarjeta de crédito, Efectivo]                                                                                         18678
[Efectivo]                                                                                                                                     12915
[Transferencia bancaria, Acordar con el comprador, Efectivo]                                                                                    2328
                                                                                                                                               ...  
[Mastercard Maestro, Cheque certificado, Transferencia bancaria, Diners, Acordar con el comprador, Giro po

In [23]:
pd.json_normalize(df.non_mercado_pago.apply(lambda x: {el: 1 for el in x})).fillna(0).add_prefix("nmp.")

,nmp.Transferencia bancaria,nmp.Acordar con el comprador,nmp.Efectivo,nmp.Tarjeta de crédito,nmp.N/a,nmp.MasterCard,nmp.Mastercard Maestro,nmp.Visa Electron,nmp.Contra reembolso,nmp.Visa,nmp.Diners,nmp.American Express,nmp.Giro postal,nmp.MercadoPago,nmp.Cheque certificado
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89996,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df = df.join(pd.json_normalize(df.pop("non_mercado_pago").apply(lambda x: {el: 1 for el in x})).fillna(0).add_prefix("nmp."))

### 3.9 Converting column shipping_mode

In [25]:
df["shipping_mode"].value_counts(dropna=False)

me2              46059
not_specified    40725
custom            3130
me1                 86
Name: shipping_mode, dtype: int64

In [26]:
df = df.join(pd.get_dummies(df.pop("shipping_mode"), prefix='shim.'))

### 3.10 Converting column warranty

In [27]:
df["warranty"] = df["warranty"].fillna("")

### 3.11 Converting column title

In [28]:
df["title"] = df["title"].fillna("")

<h2> 4. Additional transformations<h2/>

In [29]:
def remove_text_columns(df, columns):
    df = df.drop(columns, axis=1)
    return df

In [30]:
def normalize(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

In [31]:
def drop_duplicates(df):
   df = df.drop_duplicates()
   return df

In [32]:
def remove_outliers(df, columns):
   for col in columns:
      avg = df[col].mean()
      std = df[col].std()
      low = avg - 2 * std
      high = avg + 2 * std
      df = df[df[col].between(low, high, inclusive=True)]
   return df

<h2> 5. Testing all transformations<h2/>

In [34]:
df_processed = (df.
                pipe(remove_text_columns, ["warranty", "title"]).
                pipe(drop_duplicates).
                pipe(remove_outliers, ["price"]).
                pipe(normalize)
               )

df_processed

,condition,listing_type_id,price,accepts_mercadopago,automatic_relist,initial_quantity,sold_quantity,available_quantity,num_pic,num_var,...,nmp.Visa,nmp.Diners,nmp.American Express,nmp.Giro postal,nmp.MercadoPago,nmp.Cheque certificado,shim._custom,shim._me1,shim._me2,shim._not_specified
0,0.0,0.166667,0.000007,1.0,0.0,0.000000,0.000000,0.000000,0.055556,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.333333,0.000238,1.0,0.0,0.000000,0.000000,0.000000,0.166667,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.166667,0.000005,1.0,0.0,0.000000,0.000000,0.000000,0.027778,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.333333,0.000052,1.0,0.0,0.000000,0.000000,0.000000,0.055556,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.166667,0.000003,1.0,0.0,0.000000,0.000000,0.000000,0.055556,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89991,1.0,0.000000,0.000022,1.0,0.0,0.000000,0.000000,0.000000,0.083333,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
89994,0.0,0.166667,0.000108,1.0,0.0,0.000000,0.000000,0.000000,0.055556,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
89996,0.0,0.166667,0.000011,1.0,0.0,0.000000,0.000165,0.000000,0.083333,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
89998,0.0,0.166667,0.000063,1.0,0.0,0.009902,0.000000,0.009902,0.027778,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


<h2> 6. Export<h2/>

In [63]:
def convert_condition(df):
    df["condition"] = df["condition"].map({"new": 0, "used":1})
    return df

def convert_listing_type_id(df):
    df["listing_type_id"] = df["listing_type_id"].map({"free":0, "bronze":1, "silver":2, "gold":3, "gold_special":4, "gold_premium":5, "gold_pro":6})
    return df

def convert_buying_mode(df):
    df = df.join(pd.get_dummies(df.buying_mode, prefix='buy_mode.'))
    df.drop(["buying_mode"], axis=1, inplace=True)
    return df

def convert_tags(df):
    df = df.join(pd.json_normalize(df.pop("tags").apply(lambda x: {el: 1 for el in x})).fillna(0).add_prefix("tag."))
    return df
    
def convert_category_id(df):
    vc = df["category_id"].value_counts(dropna=False)
    others_list = vc[vc<400].index.to_list()
    df["category_id"] = df["category_id"].apply(lambda x: "others" if x in others_list else x)
    df = df.join(pd.get_dummies(df.pop("category_id"), prefix='cat.'))
    
    return df

def convert_category_id_test(df):
    cats = ['MLA1227', 'MLA1383', 'MLA15171', 'MLA15328', 'MLA2038', 'MLA2044', 'MLA3530', 'MLA41287']
    
    df["category_id"] = df["category_id"].apply(lambda x: "others" if x not in cats else x)
    df = df.join(pd.get_dummies(df.pop("category_id"), prefix='cat.'))
    
    return df

def convert_accepts_mercadopago(df):
    df["accepts_mercadopago"] = df["accepts_mercadopago"].apply(lambda x: int(x))
    return df

def convert_automatic_relist(df):
    df["automatic_relist"] = df["automatic_relist"].apply(lambda x: int(x))
    return df

def convert_non_mercado_pago(df):
    df = df.join(pd.json_normalize(df.pop("non_mercado_pago").apply(lambda x: {el: 1 for el in x})).fillna(0).add_prefix("nmp."))
    return df

def convert_shipping_mode(df):
    df = df.join(pd.get_dummies(df.pop("shipping_mode"), prefix='shim.'))
    return df

    
def fill_nan_texts(df):
    df["warranty"] = df["warranty"].fillna("")
    df["title"] = df["title"].fillna("")
    
def sort_columns(df):
    df = df.reindex(sorted(df.columns), axis=1)
    return df

def drop_target(df):
    df.drop(["condition"], axis=1, inplace=True)
    return df

In [72]:
def preprocess_training(df):
    df_processed = (df.
                        pipe(remove_columns, col_to_remove).
                        pipe(convert_list_into_len, col_to_len).
                        pipe(get_non_mercado_pago_info).
                        pipe(get_shipping_mode_info).
                    
                        pipe(convert_condition).
                        pipe(convert_listing_type_id).
                        pipe(convert_buying_mode).
                        pipe(convert_tags).
                        pipe(convert_category_id).
                        pipe(convert_accepts_mercadopago).
                        pipe(convert_automatic_relist).
                        pipe(convert_non_mercado_pago).
                        pipe(convert_shipping_mode).
                        pipe(sort_columns).
                    
                        pipe(remove_text_columns, ["warranty", "title"]).
                        pipe(drop_duplicates).
                        pipe(remove_outliers, ["price"]).
                        pipe(normalize)
                   )

    return df_processed

In [69]:
def preprocess_testing(df):
    df_processed = (df.
                        pipe(remove_columns, col_to_remove).
                        pipe(convert_list_into_len, col_to_len).
                        pipe(get_non_mercado_pago_info).
                        pipe(get_shipping_mode_info).

                        pipe(convert_listing_type_id).
                        pipe(convert_buying_mode).
                        pipe(convert_tags).
                        pipe(convert_category_id_test).
                        pipe(convert_accepts_mercadopago).
                        pipe(convert_automatic_relist).
                        pipe(convert_non_mercado_pago).
                        pipe(convert_shipping_mode).
                        pipe(sort_columns).
                    
                        pipe(remove_text_columns, ["warranty", "title"]).
                        pipe(normalize)
                   )

    return df_processed

In [70]:
df_train = preprocess_training(pd.DataFrame.from_records(X_train))
df_train.columns

Index(['accepts_mercadopago', 'automatic_relist', 'available_quantity',
       'buy_mode._auction', 'buy_mode._buy_it_now', 'buy_mode._classified',
       'cat._MLA1227', 'cat._MLA1383', 'cat._MLA15171', 'cat._MLA15328',
       'cat._MLA2038', 'cat._MLA2044', 'cat._MLA3530', 'cat._MLA41287',
       'cat._others', 'condition', 'initial_quantity', 'listing_type_id',
       'nmp.Acordar con el comprador', 'nmp.American Express',
       'nmp.Cheque certificado', 'nmp.Contra reembolso', 'nmp.Diners',
       'nmp.Efectivo', 'nmp.Giro postal', 'nmp.MasterCard',
       'nmp.Mastercard Maestro', 'nmp.MercadoPago', 'nmp.N/a',
       'nmp.Tarjeta de crédito', 'nmp.Transferencia bancaria', 'nmp.Visa',
       'nmp.Visa Electron', 'num_att', 'num_pic', 'num_var', 'price',
       'shim._custom', 'shim._me1', 'shim._me2', 'shim._not_specified',
       'sold_quantity', 'tag.dragged_bids_and_visits', 'tag.dragged_visits',
       'tag.free_relist', 'tag.good_quality_thumbnail',
       'tag.poor_quality_t

In [71]:
df_test = preprocess_testing(pd.DataFrame.from_records(X_test))
df_test.columns

Index(['accepts_mercadopago', 'automatic_relist', 'available_quantity',
       'buy_mode._auction', 'buy_mode._buy_it_now', 'buy_mode._classified',
       'cat._MLA1227', 'cat._MLA1383', 'cat._MLA15171', 'cat._MLA15328',
       'cat._MLA2038', 'cat._MLA2044', 'cat._MLA3530', 'cat._MLA41287',
       'cat._others', 'initial_quantity', 'listing_type_id',
       'nmp.Acordar con el comprador', 'nmp.American Express',
       'nmp.Cheque certificado', 'nmp.Contra reembolso', 'nmp.Diners',
       'nmp.Efectivo', 'nmp.Giro postal', 'nmp.MasterCard',
       'nmp.Mastercard Maestro', 'nmp.MercadoPago', 'nmp.N/a',
       'nmp.Tarjeta de crédito', 'nmp.Transferencia bancaria', 'nmp.Visa',
       'nmp.Visa Electron', 'num_att', 'num_pic', 'num_var', 'price',
       'shim._custom', 'shim._me1', 'shim._me2', 'shim._not_specified',
       'sold_quantity', 'tag.dragged_bids_and_visits', 'tag.dragged_visits',
       'tag.free_relist', 'tag.good_quality_thumbnail',
       'tag.poor_quality_thumbnail'],
 